In [1]:
import numpy as np
import pandas as pd
import os
from sklearn.model_selection import KFold, StratifiedKFold
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
import os
os.chdir('../Dataset/Train')
os.getcwd()

'/Users/benedettosimone/Desktop/CASA/LipNet-1/Dataset/Train'

In [4]:
data=[]
for l in os.listdir():
    if '.DS_Store' not in l:
        os.chdir(l)
        for m in os.listdir():
            data.append([l+'/'+m, l])
        os.chdir('..')

In [5]:
data

[['r/F01_words08_06.jpg', 'r'],
 ['r/F09_words08_07.jpg', 'r'],
 ['r/M07_words08_02.jpg', 'r'],
 ['r/M01_words08_05.jpg', 'r'],
 ['r/F08_words08_03.jpg', 'r'],
 ['r/F07_words08_01.jpg', 'r'],
 ['r/M01_words08_10.jpg', 'r'],
 ['r/M01_words08_04.jpg', 'r'],
 ['r/F08_words08_02.jpg', 'r'],
 ['r/M07_words08_03.jpg', 'r'],
 ['r/F09_words08_06.jpg', 'r'],
 ['r/M08_words08_01.jpg', 'r'],
 ['r/F01_words08_07.jpg', 'r'],
 ['r/F09_words08_04.jpg', 'r'],
 ['r/F01_words08_05.jpg', 'r'],
 ['r/M08_words08_03.jpg', 'r'],
 ['r/F09_words08_10.jpg', 'r'],
 ['r/M07_words08_01.jpg', 'r'],
 ['r/M01_words08_06.jpg', 'r'],
 ['r/F07_words08_02.jpg', 'r'],
 ['r/F07_words08_03.jpg', 'r'],
 ['r/F08_words08_01.jpg', 'r'],
 ['r/M01_words08_07.jpg', 'r'],
 ['r/F01_words08_04.jpg', 'r'],
 ['r/M08_words08_02.jpg', 'r'],
 ['r/F09_words08_05.jpg', 'r'],
 ['r/F01_words08_10.jpg', 'r'],
 ['r/F09_words08_01.jpg', 'r'],
 ['r/M08_words08_06.jpg', 'r'],
 ['r/M07_words08_04.jpg', 'r'],
 ['r/M07_words08_10.jpg', 'r'],
 ['r/M01

In [8]:
os.getcwd()

'/Users/benedettosimone/Desktop/CASA/LipNet-1/codes'

In [7]:
import csv
os.chdir('../../codes')
header = ['filename', 'label']

with open('training_labels.csv', 'w', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)

    # write the header
    writer.writerow(header)

    # write multiple rows
    writer.writerows(data)

In [9]:
train_data = pd.read_csv('training_labels.csv')
Y = train_data[['label']]

kf = KFold(n_splits = 5)

skf = StratifiedKFold(n_splits = 5, random_state = 7, shuffle = True)

In [10]:
idg = ImageDataGenerator(rescale=1./255)

In [11]:
def get_model_name(k):
    return 'model_'+str(k)+'.h5'

In [17]:
def create_new_model():
    import os
    import numpy as np
    from keras.models import Sequential # To initialise the nn as a sequence of layers
    from keras.layers import Convolution2D # To make the convolution layer for 2D images
    from keras.layers import MaxPooling2D #
    from keras.layers import Flatten
    from keras.layers import Dense
    from keras.layers import Dropout
    from keras.callbacks import CSVLogger
    from tensorflow.keras.optimizers import RMSprop
    from keras.layers import BatchNormalization
    from tensorflow.keras.optimizers import Adam
    from keras.models import load_model
    from keras.callbacks import ModelCheckpoint
    from keras.layers import Activation
    from keras.layers import GlobalAveragePooling2D

    
    csv = CSVLogger("2_adam.log")
    #filepath="weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"
    #checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
    # Initialising the CNN
    classifier = Sequential()

    # Step 1 - Convolution
    classifier.add(Convolution2D(32, (2, 2), input_shape=(224, 224, 1), activation='relu', strides=2, name='convo1'))
    classifier.add(Convolution2D(64, (3, 3), activation='relu', name='convo2'))
    # Step 1 - Pooling
    classifier.add(MaxPooling2D(pool_size=(2, 2)))

    # Step 2 - Convolution
    classifier.add(Convolution2D(64, (3, 3), activation='relu', name='convo3'))
    # Step 2 - Pooling
    classifier.add(MaxPooling2D(pool_size=(2, 2)))
    # Step 3 - Convolution
    classifier.add(Convolution2D(64, (3, 3), activation='relu', name='convo4'))
    # Step 3 - Pooling
    classifier.add(MaxPooling2D(pool_size=(2, 2)))
    #Step 4 - Flattening

    classifier.add(Flatten())

    classifier.add(BatchNormalization())
    classifier.add(Dropout((0.5)))
    classifier.add(Dense(1024, activation='relu'))

    '''classifier.add(BatchNormalization())
    classifier.add(Dropout((0.5)))
    classifier.add(Dense(512, activation = 'relu'))
    '''

    classifier.add(BatchNormalization())
    classifier.add(Dropout((0.4)))
    classifier.add(Dense(20, activation='softmax'))

    return classifier




In [22]:
VALIDATION_ACCURACY = []
VALIDATION_LOSS = []

save_dir = 'saved_models/'
fold_var = 1

for train_index, val_index in skf.split(np.zeros(2602),Y):
    training_data = train_data.iloc[train_index]
    validation_data = train_data.iloc[val_index]

    train_data_generator = idg.flow_from_dataframe(training_data, directory = '../Dataset/Train',
                               x_col = "filename", y_col = "label",
                               class_mode = "categorical", shuffle = True, color_mode="grayscale",target_size=(224,224),batchsize=32)
    valid_data_generator  = idg.flow_from_dataframe(validation_data, directory =  '../Dataset/Train',
                            x_col = "filename", y_col = "label",
                            class_mode = "categorical", shuffle = True, color_mode="grayscale",target_size=(224,224),batchsize=32)

    # CREATE NEW MODEL
    model = create_new_model()
    # COMPILE NEW MODEL
    model.compile(loss='categorical_crossentropy',
              optimizer="Adam",
              metrics=['accuracy'])

    # CREATE CALLBACKS
    checkpoint = tf.keras.callbacks.ModelCheckpoint(save_dir+get_model_name(fold_var),
                            monitor='val_accuracy', verbose=1,
                            save_best_only=True, mode='max')
    callbacks_list = [checkpoint]
    # There can be other callbacks, but just showing one because it involves the model name
    # This saves the best model
    # FIT THE MODEL
    history = model.fit(train_data_generator,
                epochs=10,
                callbacks=callbacks_list,
                validation_data=valid_data_generator)
    #PLOT HISTORY
    #:
    #:

    # LOAD BEST MODEL to evaluate the performance of the model
    model.load_weights("saved_models/model_"+str(fold_var)+".h5")
    print('Sto salvando')

    results = model.evaluate(valid_data_generator)
    results = dict(zip(model.metrics_names,results))

    VALIDATION_ACCURACY.append(results['accuracy'])
    VALIDATION_LOSS.append(results['loss'])

    tf.keras.backend.clear_session()

    fold_var += 1

/opt/anaconda3/lib/python3.9/site-packages/keras_preprocessing/image/dataframe_iterator.py:279: UserWarning: Found 1 invalid image filename(s) in x_col="filename". These filename(s) will be ignored.
  warnings.warn(


Found 2080 validated image filenames belonging to 20 classes.
Found 520 validated image filenames belonging to 20 classes.
Epoch 1/10
65/65 [==============================] - ETA: 0s - loss: 3.3261 - accuracy: 0.1712
Epoch 00001: val_accuracy improved from -inf to 0.11731, saving model to saved_models/model_1.h5
65/65 [==============================] - 54s 816ms/step - loss: 3.3261 - accuracy: 0.1712 - val_loss: 2.9208 - val_accuracy: 0.1173
Epoch 2/10
65/65 [==============================] - ETA: 0s - loss: 2.0510 - accuracy: 0.4202
Epoch 00002: val_accuracy did not improve from 0.11731
65/65 [==============================] - 54s 824ms/step - loss: 2.0510 - accuracy: 0.4202 - val_loss: 2.9475 - val_accuracy: 0.1173
Epoch 3/10
65/65 [==============================] - ETA: 0s - loss: 1.4017 - accuracy: 0.5798
Epoch 00003: val_accuracy improved from 0.11731 to 0.19038, saving model to saved_models/model_1.h5
65/65 [==============================] - 53s 815ms/step - loss: 1.4017 - accura

/opt/anaconda3/lib/python3.9/site-packages/keras_preprocessing/image/dataframe_iterator.py:279: UserWarning: Found 2 invalid image filename(s) in x_col="filename". These filename(s) will be ignored.
  warnings.warn(


Epoch 1/10
65/65 [==============================] - ETA: 0s - loss: 3.3202 - accuracy: 0.1592
Epoch 00001: val_accuracy improved from -inf to 0.06718, saving model to saved_models/model_2.h5
65/65 [==============================] - 62s 934ms/step - loss: 3.3202 - accuracy: 0.1592 - val_loss: 2.9400 - val_accuracy: 0.0672
Epoch 2/10
65/65 [==============================] - ETA: 0s - loss: 2.2890 - accuracy: 0.3646
Epoch 00002: val_accuracy did not improve from 0.06718
65/65 [==============================] - 64s 985ms/step - loss: 2.2890 - accuracy: 0.3646 - val_loss: 2.9709 - val_accuracy: 0.0595
Epoch 3/10
65/65 [==============================] - ETA: 0s - loss: 1.6130 - accuracy: 0.5402
Epoch 00003: val_accuracy improved from 0.06718 to 0.12284, saving model to saved_models/model_2.h5
65/65 [==============================] - 56s 859ms/step - loss: 1.6130 - accuracy: 0.5402 - val_loss: 2.7501 - val_accuracy: 0.1228
Epoch 4/10
65/65 [==============================] - ETA: 0s - loss: 1.

In [26]:
VALIDATION_ACCURACY

[0.6461538672447205,
 0.6410748362541199,
 0.6692307591438293,
 0.5799614787101746,
 0.6096153855323792]

In [28]:

VALIDATION_LOSS

[1.0657050609588623,
 1.135345697402954,
 1.206666350364685,
 1.4198025465011597,
 1.3281400203704834]